In [4]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()


In [5]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [8]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price']=housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [10]:
from urllib.parse import urlparse

X=data.drop(columns='Price')
y=data['Price']

In [21]:
def HyperparamterTuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_searchcv=GridSearchCV(estimator=rf,param_grid=param_grid,cv=3,scoring="neg_mean_absolute_error",n_jobs=-1,verbose=True)
    grid_searchcv.fit(X_train,y_train)
    return grid_searchcv

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

from mlflow.models import infer_signature
signature=infer_signature(X_train,y_test)

param_grid={
    "n_estimators":[100,200],
    "max_depth":[5,10,None],
    "min_samples_leaf":[2,5],
    "max_leaf_nodes":[1,2]
}
with mlflow.start_run():
    grid_search=HyperparamterTuning(X_train,y_train,param_grid)
    best_model=grid_search.best_estimator_
    y_pred=best_model.predict(X_test)
    mse=mean_absolute_error(y_test,y_pred)
    mlflow.log_param('Best_n_estimators',grid_search.best_params_['n_estimators'])
    mlflow.log_param('max_depth',grid_search.best_params_['max_depth'])
    mlflow.log_param('min_samples_leaf',grid_search.best_params_['min_samples_leaf'])
    mlflow.log_param('max_leaf_nodes',grid_search.best_params_['max_leaf_nodes'])
    mlflow.log_metric('metrics',mse)

    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_uri_type_store!='file':
        mlflow.sklearn.log_model(best_model,'model',registered_model_name='Best_RandomForest Model')

    else:
        mlflow.sklearn.log_model(best_model,'model',signature=signature)
    
    print("hyperparamter",grid_search.best_params_)
    print("mse",mse)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


f:\MLops\MLFlowStater\.venv\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
36 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "f:\MLops\MLFlowStater\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "f:\MLops\MLFlowStater\.venv\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "f:\MLops\MLFlowStater\.venv\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "f:\MLops\MLFlowStater\.venv\lib

hyperparamter {'max_depth': 5, 'max_leaf_nodes': 2, 'min_samples_leaf': 5, 'n_estimators': 100}
mse 0.7422964450723002
